# Wprowadzenie

## Interface

Unity składa się z kilku podstawowych elementów. Na środku mamy **Scenę** - naszą przestrzeń na której układamy obiekty gry. Możemy jednocześnie przejść do **Gry**, by podejrzeć jak faktycznie będzie wyglądał poziom dla końcowego użytkownika. Tutaj mamy ograniczony ruch do podglądu z **Kamery** - obiektu odpowiedzialnego za przetwarzanie obrazu i dźwięku z systemu do gracza.

Po lewej stronie znajduje się lista obiektów na scenie. Jest to bardzo przydatny układ, bo nawet przy kilkunastu obiektach możemy się już zagubić, podczas gdy lista pozwala nam w prosty sposób odnaleźć każdy z elementów. Scen może być więcej niż jedna, ale na raz jesteśmy w stanie podejrzeć i edytować tylko jedną z nich.

Z prawej strony posiadamy **Inspektor** obiektów. Każdy obiekt jaki stworzymy - pomimo najróżniejszych funkcji i parametrów - fundamentalnie jest po prostu **Obiektem Gry** (w oryginale `GameObject`) wyposażonym w różne dodatki. Część z tych dodatków jest dostarczana przez samo środowisko, część natomiast zaprojektujemy sami w środowisku Visual Studio w języku `C#`.

Na dole mamy okno **Projektu** - w nim mamy podgląd wszystkich dostępnych dla nas obiektów i narzędzi, nie tylko aktualnie wykorzystywanych na scenie. O ile o reszcie edytora możemy myśleć jak o projektowanej scenie z aktorami, o tyle okno Projektu jest naszym przybornikiem z narzędziami.

## Gra w Ponga

![](https://upload.wikimedia.org/wikipedia/commons/6/62/Pong_Game_Test2.gif)

Pong jest historycznie uznawany za najstarszą grę video. Pierwszy raz został stworzony w 1958 roku na oscyloskopie (pod nazwą Tennis for Two). W 1972 roku został oficjalnie wydany przez firmę Atari. Zaprojektowanie tej gry od zera, na urządzeniach pierwotnie służących kompletnie innym celom, było wyzwaniem - dla nas jednak będzie to bardzo proste pierwsze ćwiczenie obsługi edytora.

![](https://teachingkidstocode.io/wp-content/uploads/2017/08/Pong-1920x1200-31.jpg)

Po pierwsze, musimy myślowo rozłożyć koncept który posiadamy w niskiej rozdzielczości w naszych głowach i przeanalizować go, by być w stanie go dokładnie odtworzyć. W Pongu możemy wyróżnić następujące elementy:

- Paletki - bloczki kontrolowane przez graczy, możliwe do przesuwania w górę i w dół po planszy. Owe bloczki muszą być ograniczone przez planszę wyświetlaną na ekranie. Paletki muszą również stanowić element kolizji dla naszej piłki.

- Piłka - kolejny bloczek który podlega prawom fizyki. Piłka musi odbijać się od brzegów planszy (dyskusyjne) i od paletek (absolutnie wymagane). Piłka również powinna mieć zawsze minimalną prędkość, by nie spotkać się z sytuacją, w której czekamy w nieskończoność, bo piłka zatrzymała się na środku ekranu. Po wyleceniu piłki poza granice planszy, dodawany jest punkt jednemu z graczy i gra zaczyna się od początku.

- Punkty - napis wyświetlany na ekranie, informujący graczy o aktualnym wyniku gry. W przypadku wylecenia piłki poza ekran, punkty powinny być aktualizowane do nowej wartości.

### Proces tworzenia - kamera

W pierwszej kolejności musimy ustawić nasz ekran gry. Unity pozwala na ekran typu `Free aspect`, dostosowujący się do ekranu o dowolnej wielkości, odkryjemy jednak że zapewnienie tej swobody będzie kosztowało nas sporo pracy. Umówimy się zatem na proporcje 16:9, odpowiadające aktualnemu standardowi FullHD - 1920:1080. Tę relację ustalimy w ekranie Gry, tuż obok napisu Display 1.

Następnie, kamera. Możliwość obserwacji gry jest krytyczna dla odbiorcy *[potrzebne cytowanie]*. Standardowa scena dostarcza nam od razu obiekt **Main Camera**, co oszczędza nam dodatkowych kliknięć.

W nowoczesnych grach najczęściej korzysta się z rzutu perspektywistycznego, tj. obrazu generowanego z kamery mającej swoje fizyczne położenie w przestrzeni i kąt widzenia.

Starsze gry i dzisiejsze bardziej abstrakcyjne produkcje korzystają natomast z rzutu **ortograficznego**, który zakłada położenie kamery w nieskończonej odległości, oddając obserwatorowi konkretny wycinek przestrzeni. Jak można się domyślać, perspektywa jest w takim przypadku mocno zaburzona, przy jednoczesnym zwiększeniu przejrzystości dwuwymiarowego układu.

Dobre porównanie obydwu rzutów daje następujący obrazek:

![](https://www.script-tutorials.com/demos/456/4.png)

### Proces tworzenia - paletki

Stworzymy teraz paletkę, tj. wydłużony klocuch (Cube). Do zestawu komponentów dorzucimy nasz autorski skrypt, który nazwiemy `PlayerScript`:

```csharp
using System.Collections;
using System.Collections.Generic;
using UnityEngine;

public class PlayerScript : MonoBehaviour
{
    // Start is called before the first frame update
    void Start()
    {
        
    }

    public float MaxRange = 18;
    public float PlayerSpeed = 1;
    public string InputName = "Player 1";

    // Update is called once per frame
    void Update()
    {
        var pos = transform.position;
        pos.y += Input.GetAxis(InputName) * PlayerSpeed;
        if (pos.y > MaxRange) pos.y = MaxRange;
        if (pos.y < -MaxRange) pos.y = -MaxRange;
        transform.position = pos;
    }
}
```

Powyższy skrypt odpowiada za ruch paletki w górę i w dół zależnie od wciśniętych przez gracza klawiszy. Nie jest tu przedstawione *jakie* to klawisze - to ustalimy w **Project Settings**, zakładce **Inputs**. Stąd też zmienna odpowiadajaca za nazwę tych inputów - domyślnie będzie to `Player 1`, ze względu jednak na publiczność tej zmiennej, będziemy mogli zmienić tę nazwę wedle naszej woli.

Zmienna `MaxRange` ma odpowiadać za maksymalne (i minimalne) wychylenie paletki w pionie. `PlayerSpeed` ma dodatkowo pozwalać na modyfikację prędkości paletki.

Po dodaniu zmian, zapiszemy naszą paletkę jako **Prefab** - spreparowany (sprefabrykowany?) wcześniej skomplikowany obiekt, możliwy do skopiowania z naszej skrzynki z narzędziami.

### Proces tworzenia - piłka

Przyszedł czas na piłkę - stworzymy nowy obiekt, może to być klocek, może być sfera, wedle uznania. Naszej piłce musimy dodać komponent **Rigidbody** - podzespół odpowiedzialny za fizyczne zachowanie się elementu. Przezornie wyłączymy w nim grawitację, zamrozimy również ruch w osi `z` - bez tego nie będziemy mogli traktować naszego układu jak w pełni 2d.

Dodatkowo, dodamy mu jeszcze podzespół skryptu, który nazwiemy `BallScript`:

```csharp
using System.Collections;
using System.Collections.Generic;
using UnityEngine;
using UnityEngine.UI;

public class BallScript : MonoBehaviour
{
    public Vector3 InitialVelocity = new Vector3(1,0.01f,0);
    public float MinimalVelocity = 10;
    public float MaxRange = 40;


    // Start is called before the first frame update
    void Start()
    {
        started = false;
    }

    private bool started = false;

    // Update is called once per frame
    void Update()
    {
        var physics = GetComponent<Rigidbody>();

        if (!started && Input.GetButton("Start"))
        {
            started = true;
            physics.velocity = InitialVelocity * MinimalVelocity;
            
            var StartText = GameObject.Find("Start Text");
            if (StartText)
            {
                StartText.GetComponent<Text>().text = "";
            }
        }

        if (physics.velocity.magnitude < MinimalVelocity)
        {
            physics.velocity = physics.velocity.normalized * MinimalVelocity;
        }

        var managerElement = GameObject.Find("Game Manager");
        
        var x = transform.position.x;
        if (x > MaxRange)
        {
            if (managerElement)
            {
                var Manager = managerElement.GetComponent<GameManagerScript>();
                Manager.AddPoint("Player 2");
            }
            NewBall(-1);
        }

        if (x < -MaxRange)
        {
            if (managerElement)
            {
                var Manager = managerElement.GetComponent<GameManagerScript>();
                Manager.AddPoint("Player 1");
            }
            NewBall(+1);
        }


    }

    void NewBall(int Direction)
    {
        Destroy(gameObject);
        var StartText = GameObject.Find("Start Text");
        if (StartText)
        {
            StartText.GetComponent<Text>().text = "PRESS SPACE TO START";
        }
        var element = Instantiate(gameObject, Vector3.zero, Quaternion.identity);
        element.GetComponent<BallScript>().enabled = true;
        element.GetComponent<BallScript>().InitialVelocity = new Vector3(1*Direction, 0.01f, 0);
    }
}
```

### Proces tworzenia - Game Manager

Po skomplikowanym procesie programowania piłki (którego Wam oszczędziłem, publikując gotowy skrypt), należy dodać małe uzupełnienia. Potrzebujemy obiektu `Game Manager`, który będzie przechowywał bieżący wynik gry, oraz zarządzał zmianę wyświetlanych punktów. Stworzymy pusty obiekt o wyżej wspomnianej nazwie, zawierający skrypt `GameManagerScript`.

```csharp
using System.Collections;
using System.Collections.Generic;
using UnityEngine;
using UnityEngine.UI;

public class GameManagerScript : MonoBehaviour
{
    // Start is called before the first frame update
    void Start()
    {
        
    }

    // Update is called once per frame
    void Update()
    {
        
    }

    public int Score1 = 0;
    public int Score2 = 0;

    public void AddPoint(string player)
    {
        if (player == "Player 1")
        {
            Score1++;
            var ScoreA = GameObject.Find("Player 1 Score");
            if (ScoreA)
            {
                ScoreA.GetComponent<Text>().text = $"{Score1}";
            }
        }
        if (player == "Player 2")
        {
            Score2++;
            var ScoreB = GameObject.Find("Player 2 Score");
            if (ScoreB)
            {
                ScoreB.GetComponent<Text>().text = $"{Score2}";
            }
        }
    }
}
```

### Proces tworzenia - Tekst wyników, tekst początkowy

Ostatni element - dodamy teksty punktów oraz tekst wyświetlany na początku gry (informujący o wymogu wciśnięciu spacji). Do Main Camera dodamy element `Text`, który przeniesie nas do przestrzeni Canvas - elementu gry istniejącego bezpośrednio na powierzchni ekranu, nie biorącego udziału w faktycznej przestrzeni gry.